# 1	DATA WRANGLING

Import liberaries

In [1]:
import pandas as pd
import requests
import json
import tweepy
import datetime
import os

## 1.1 Gathering Data

### 1.1.1 Load data files

#### 1.1.1.1 Loading Enhanced Twitter Archive Data

In [2]:
df_arch = pd.read_csv('twitter-archive-enhanced.csv')
#df_arch.head()

#### 1.1.1.2 Loading Image Predictions Data

In [3]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/\
August/599fd2ad_image-predictions/image-predictions.tsv'
file_name = url.split('/')[-1]
#print(file_name)
r_predict = requests.get(url)
status = r_predict.status_code
#print(status)
if not os.path.isfile(file_name):
    with open(file_name, 'wb') as file:
        file.write(r_predict.content)

df_predict = pd.read_csv(file_name, sep = '\t')
#df_predict.head()

#### 1.1.1.3 Loading Twitter API Data

In [4]:
#from tweepy import OAuthHandler
#from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions

#consumer_key = 'HIDDEN'
#consumer_secret = 'HIDDEN'
#access_token = 'HIDDEN'
#access_secret = 'HIDDEN'

#auth = OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)

#api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API

#tweet_ids = df_arch.tweet_id.values
#len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive

#count = 0
#fails_dict = {}
#start = timer()
# Save each tweet's returned JSON as a new line in a .txt file

#if not os.path.isfile('tweet_json.txt'):
#    with open('tweet_json.txt', 'w') as outfile:
        # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
#        for tweet_id in tweet_ids:
#            count += 1
            #print(str(count) + ": " + str(tweet_id))
#            try:
#                tweet = api.get_status(tweet_id, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, tweet_mode='extended')
#                print("Success")
#                json.dump(tweet._json, outfile)
#                outfile.write('\n')
#            except tweepy.TweepError as e:
#                print("Fail")
#                fails_dict[tweet_id] = e
#                pass
#end = timer()
#print(end - start)
#print(fails_dict)

In [5]:
df_list = []
with open('tweet-json.txt', 'r') as file:
    for line in file:
        tweeta = json.loads(line)
        #keys = tweeta.keys
        #print(tweeta)
        tweet_id = tweeta['id']
        retweets_count = tweeta['retweet_count']
        favorite_count = tweeta['favorite_count']
        followers_count = tweeta['user']['followers_count']
        df_list.append({'tweet_id':tweet_id,
                       'retweet_count':retweets_count,
                       'favorite_count':favorite_count,
                       'followers_count':followers_count})
        
df_api = pd.DataFrame(df_list)
#df_api.to_csv('fff.csv', index=False)
#df_api.head(20)

## 1.2 Assessing Data

### 1.2.1 The Twitter Archive Table

#### 1.2.1.1 Visual Assessment

In [6]:
df_arch.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1513,691096613310316544,NaN,NaN,2016-01-24 03:14:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we see one dog giving a puptalk to anothe...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/691096613...,11,10,None,None,None,None,None
647,793135492858580992,NaN,NaN,2016-10-31 17:00:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Your favorite squad is looking extra h*ckin sp...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/793135492...,13,10,None,None,None,None,None
147,863432100342583297,NaN,NaN,2017-05-13 16:33:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Belle. She's never been more pupset. E...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863432100...,12,10,Belle,None,None,None,None
1633,684241637099323392,NaN,NaN,2016-01-05 05:14:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Obi. He got camera shy. 12/10 https://...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/684241637...,12,10,Obi,None,None,None,None
1275,709179584944730112,NaN,NaN,2016-03-14 00:49:23 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Sammy. He's in a tree. Very excited ab...,NaN,NaN,NaN,https://vine.co/v/iwAjdlEjwMl,13,10,Sammy,None,None,None,None


* Column 'name' has invalid entries 'None' instead of NaN
* Column 'expanded_urls' has missing data
* Columns (doggo, floofer, pupper, puppo) have names that should be values not names

#### 1.2.1.2 Programmatic Assessment

In [7]:
df_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [8]:
df_arch[df_arch.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


* column tweet_id should be of type string, or object here
* column timestamp should have type datetime

### 1.2.2 The Image Predictions Table

#### 1.2.2.1 Visual Assessment

In [9]:
df_predict.tail(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2074,892420643555336193,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False


* columns p1, p2, p3 and their relatives have unclear names

#### 1.2.2.2 Programmatic Assessment

In [10]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [11]:
df_predict[df_predict.duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


* number of rows is 2075 instead of 2356 in archive table

### 1.2.1 The Twitter API Table

#### 1.2.3.1 Visual Assessment

In [12]:
df_api.sample(5)

,tweet_id,retweet_count,favorite_count,followers_count
1897,674670581682434048,729,1751,3201004
895,759047813560868866,2302,7227,3200908
1445,696488710901260288,1166,2788,3200950
1124,729854734790754305,1268,4396,3200944
25,886983233522544640,8084,35859,3200890


* The three tables have data that are related to the tweets, and these data should be in one table as all tables concern with the tweets’ stats and info

#### 1.2.3.2 Programmatic Assessment

In [13]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   tweet_id         2354 non-null   int64
 1   retweet_count    2354 non-null   int64
 2   favorite_count   2354 non-null   int64
 3   followers_count  2354 non-null   int64
dtypes: int64(4)
memory usage: 73.7 KB


In [14]:
df_api[df_api.duplicated()]

,tweet_id,retweet_count,favorite_count,followers_count


* number of rows is 2354 instead of 2356 in archive table

### 1.2.4 Assessment Summary

#### 1.2.4.1 Quality Issues

1.	Twitter Archive Table: Column 'name' has invalid entries 'None' instead of NaN (Accuracy)
2.	Twitter Archive Table: column tweet_id should be of type string, or object here (Consistency)
3.	Twitter Archive Table: column timestamp should have type datetime (Consistency)
4.	Image Predictions Table: column tweet_id should have type string (Completeness)
5.	Twitter Archive Table: Column 'expanded_urls' has missing data (Consistency)
6.	Image Predictions Table: number of rows is 2075 instead of 2356 in archive table (Completeness)
7.	Twitter API Table: number of rows is 2354 instead of 2075 in image predictions tabl (Completeness)
8.	Twitter Archive Table: table includes retweets
9.	Twitter Archive Table: remove ALL NaN columns (Consistency)
10.	Image Predictions Table: non-descriptive columns' names, p1, p2 ,p3...

#### 1.2.4.1 Tideness Issues

1.	Twitter Archive Table: Columns names of 'doggo', 'floofer', 'pupper', and 'puppo' are actually values
2.	Clean the Dataset from tweets that have no images
3.	The three tables have data that are related to the tweets, and these data should be in one table as all tables concern with the tweets’ stats and info

## 1.3 Cleaning Data

Copying data tables

In [15]:
df_arch_clean = df_arch.copy()
df_predict_clean = df_predict.copy()
df_api_clean = df_api.copy()

### 1.3.1 Twitter Archive Table: Column 'name' has invalid entries 'None' instead of NaN

### Define
Replace the 'None' value in 'name' column with NaN
### Code

In [16]:
df_arch_clean.name.value_counts()

None       745
a           55
Charlie     12
Cooper      11
Oliver      11
          ... 
Adele        1
Bowie        1
Kramer       1
Beya         1
Covach       1
Name: name, Length: 957, dtype: int64

In [17]:
df_arch_clean['name'].replace('None', 'NaN', inplace = True)

### Test

In [18]:
df_arch_clean.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1189,718246886998687744,NaN,NaN,2016-04-08 01:19:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Alexanderson. He's got a weird ass bir...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/718246886...,3,10,Alexanderson,None,None,None,None
1495,692535307825213440,NaN,NaN,2016-01-28 02:30:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Amber. She's a Fetty Woof. 10/10 would...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/692535307...,10,10,Amber,None,None,None,None
1352,703774238772166656,NaN,NaN,2016-02-28 02:50:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""YOU CAN'T HANDLE THE TRUTH"" both 10/10 https:...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/703774238...,10,10,NaN,None,None,None,None
737,780858289093574656,NaN,NaN,2016-09-27 19:54:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dash. He's very stylish, but also incr...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/780858289...,10,10,Dash,None,None,None,None
1651,683498322573824003,NaN,NaN,2016-01-03 04:01:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Griffin. He's desperate for both a phy...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/683498322...,11,10,Griffin,None,None,None,None
742,780476555013349377,NaN,NaN,2016-09-26 18:38:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Patreon: Well. @dog_rates is on Patreon. \...,7.804657e+17,1.228326e+09,2016-09-26 17:55:00 +0000,"https://www.patreon.com/WeRateDogs,https://twi...",12,10,NaN,None,None,None,None
496,813157409116065792,NaN,NaN,2016-12-25 23:00:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Layla. It is her first Christmas. She ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/813157409...,12,10,Layla,None,None,None,None
524,809220051211603969,NaN,NaN,2016-12-15 02:14:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kyro. He's a Stratocumulus Flop. Tongu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/809220051...,12,10,Kyro,None,None,None,None
354,831262627380748289,NaN,NaN,2017-02-13 22:03:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tucker. He's feeling h*ckin festive an...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/831262627...,12,10,Tucker,None,None,None,None
1248,711363825979756544,NaN,NaN,2016-03-20 01:28:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Please, no puparazzi"" 11/10 https://t.co/nJIX...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/711363825...,11,10,NaN,None,None,None,None


In [19]:
df_arch_clean.name.value_counts()

NaN        745
a           55
Charlie     12
Cooper      11
Oliver      11
          ... 
Adele        1
Hero         1
Bowie        1
Beya         1
Fillup       1
Name: name, Length: 957, dtype: int64

compared to section 1.2.1.1 above, now the 'name' column has no 'none' values, instead it has 'NaN' values.

### 1.3.2 Twitter Archive Table: column tweet_id should be of type string, or object here

### Define
Convert the data type of the 'tweet_id' column values to be string
### Code

In [20]:
df_arch_clean.info()
df_arch_clean['tweet_id'] = df_arch_clean['tweet_id'].astype('object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

### Test

In [21]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   object 
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

now the 'tweet_id' column has 'object' type not 'int64'.

### 1.3.3 Twitter Archive Table: column timestamp should have type datetime

### Define
Convert the data type of the 'timestamp' column values to be datetime
### Code

In [22]:
df_arch_clean.timestamp = df_arch_clean.timestamp.astype('datetime64[ns, UTC]')

### Test

In [23]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   object             
 1   in_reply_to_status_id       78 non-null     float64            
 2   in_reply_to_user_id         78 non-null     float64            
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    float64            
 7   retweeted_status_user_id    181 non-null    float64            
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

now the 'timestamp' column has 'datetime64' type not 'object'.

### 1.3.4 Image Predictions Table: column tweet_id  should have type string

### Define
Convert the data type of the 'tweet_id' column values to be string
### Code

In [24]:
df_predict_clean.info()
df_predict_clean['tweet_id'] = df_predict_clean['tweet_id'].astype('object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### Test

In [25]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   object 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


now the 'tweet_id' column has 'object' type not 'int64'.

### 1.3.5 Twitter Archive Table: Column 'expanded_urls' has missing data
### Define
Drop rows with missing values in the 'expanded_urls' field
### Code

In [26]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   object             
 1   in_reply_to_status_id       78 non-null     float64            
 2   in_reply_to_user_id         78 non-null     float64            
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    float64            
 7   retweeted_status_user_id    181 non-null    float64            
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

In [27]:
df_arch_clean.dropna(subset=['expanded_urls'], inplace = True)

### Test

In [28]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2297 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2297 non-null   object             
 1   in_reply_to_status_id       23 non-null     float64            
 2   in_reply_to_user_id         23 non-null     float64            
 3   timestamp                   2297 non-null   datetime64[ns, UTC]
 4   source                      2297 non-null   object             
 5   text                        2297 non-null   object             
 6   retweeted_status_id         180 non-null    float64            
 7   retweeted_status_user_id    180 non-null    float64            
 8   retweeted_status_timestamp  180 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2297 non-null   int64           

Now, the number of row in the table is 2297 instead of 2356 after dropping the tweets that have no images.

### 1.3.6 Image Predictions Table: number of rows is 2075 instead of 2356 in archive table
### Define
Drop rows that are not existant in both tables
### Code

In [29]:
image_index = df_predict_clean.tweet_id
df_arch_clean.info()
#image_index[-5:]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2297 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2297 non-null   object             
 1   in_reply_to_status_id       23 non-null     float64            
 2   in_reply_to_user_id         23 non-null     float64            
 3   timestamp                   2297 non-null   datetime64[ns, UTC]
 4   source                      2297 non-null   object             
 5   text                        2297 non-null   object             
 6   retweeted_status_id         180 non-null    float64            
 7   retweeted_status_user_id    180 non-null    float64            
 8   retweeted_status_timestamp  180 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2297 non-null   int64           

In [30]:
df_arch_clean = df_arch_clean[df_arch_clean['tweet_id'].isin(image_index)]

### Test

In [31]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2075 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2075 non-null   object             
 1   in_reply_to_status_id       23 non-null     float64            
 2   in_reply_to_user_id         23 non-null     float64            
 3   timestamp                   2075 non-null   datetime64[ns, UTC]
 4   source                      2075 non-null   object             
 5   text                        2075 non-null   object             
 6   retweeted_status_id         81 non-null     float64            
 7   retweeted_status_user_id    81 non-null     float64            
 8   retweeted_status_timestamp  81 non-null     object             
 9   expanded_urls               2075 non-null   object             
 10  rating_numerator            2075 non-null   int64           

Now, the number of row in the table is 2075 instead of 2356 after dropping the tweets that have no predictions for images.

### 1.3.7 Twitter API Table: number of rows is 2354 instead of 2075 in image predictions table
### Define
Drop rows that are not existant in both tables
### Code

In [32]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   tweet_id         2354 non-null   int64
 1   retweet_count    2354 non-null   int64
 2   favorite_count   2354 non-null   int64
 3   followers_count  2354 non-null   int64
dtypes: int64(4)
memory usage: 73.7 KB


In [33]:
df_api_clean = df_api_clean[df_api_clean['tweet_id'].isin(image_index)]

In [34]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2353
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   tweet_id         2073 non-null   int64
 1   retweet_count    2073 non-null   int64
 2   favorite_count   2073 non-null   int64
 3   followers_count  2073 non-null   int64
dtypes: int64(4)
memory usage: 81.0 KB


Now, the number of row in the table is 2073 instead of 2075 after dropping the tweets that are not in the archive table.

### 1.3.8 Twitter Archive Table: table includes retweets
### Define
Drop rows that are retweets not original tweets. Drop columns for retweets.
### Code

In [35]:
#df_arch_clean['in_reply_to_status_id'].value_counts(dropna=False)

In [36]:
df_arch_clean['retweeted_status_id'].value_counts(dropna=False)

NaN             1994
7.424232e+17       1
6.798284e+17       1
8.224891e+17       1
7.638376e+17       1
                ... 
6.675484e+17       1
7.902771e+17       1
7.761133e+17       1
8.479710e+17       1
8.222448e+17       1
Name: retweeted_status_id, Length: 82, dtype: int64

In [37]:
NaN_list = ['NaN','NaN']
#df_arch_clean = df_arch_clean[df_arch_clean['in_reply_to_status_id'].isin(NaN_list)]
df_arch_clean = df_arch_clean[df_arch_clean['retweeted_status_id'].isin(NaN_list)]

In [38]:
df_arch_clean.drop(columns=['retweeted_status_id'], inplace=True)
#df_arch_clean.drop(columns=['in_reply_to_status_id','retweeted_status_id'], inplace=True)

### Test

In [39]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    1994 non-null   object             
 1   in_reply_to_status_id       23 non-null     float64            
 2   in_reply_to_user_id         23 non-null     float64            
 3   timestamp                   1994 non-null   datetime64[ns, UTC]
 4   source                      1994 non-null   object             
 5   text                        1994 non-null   object             
 6   retweeted_status_user_id    0 non-null      float64            
 7   retweeted_status_timestamp  0 non-null      object             
 8   expanded_urls               1994 non-null   object             
 9   rating_numerator            1994 non-null   int64              
 10  rating_denominator          1994 non-null   int64           

Now, the number of row in the table is 1994 instead of 2075 after dropping the tweets that are retweets. The column 'retweeted_status_id' has been dropped.

### 1.3.9 Twitter Archive Table: Remove columns with ‘NaN’ in data fields 
### Define
Columns 'retweeted_status_user_id' and 'retweeted_status_timestamp' have no data. Drop columns that have no data.
### Code

In [40]:
df_arch_clean = df_arch_clean.dropna(axis=1, how='all')

### Test

In [41]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               1994 non-null   object             
 1   in_reply_to_status_id  23 non-null     float64            
 2   in_reply_to_user_id    23 non-null     float64            
 3   timestamp              1994 non-null   datetime64[ns, UTC]
 4   source                 1994 non-null   object             
 5   text                   1994 non-null   object             
 6   expanded_urls          1994 non-null   object             
 7   rating_numerator       1994 non-null   int64              
 8   rating_denominator     1994 non-null   int64              
 9   name                   1994 non-null   object             
 10  doggo                  1994 non-null   object             
 11  floofer                1994 non-null   object           

For the sake of cleaning the archive table, the columns with no data have been dropped; namely 'retweeted_status_user_id' and 'retweeted_status_timestamp', and hence the number of columns is 14 only.

### 1.3.10 Image Predictions Table: non-descriptive columns' names
### Define
Rename p1, p2, and p3 columns to more descriptive names.
### Code

In [42]:
df_predict_clean.rename(columns={"p1": "Prediction_P1", "p2": "Prediction_P2", "p3": "Prediction_P3"}, inplace=True)
df_predict_clean.rename(columns={"p1_conf": "Confidence_of_P1", "p2_conf": "Confidence_of_P2", "p3_conf": "Confidence_of_P3"}, inplace=True)

### Test

In [43]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_id          2075 non-null   object 
 1   jpg_url           2075 non-null   object 
 2   img_num           2075 non-null   int64  
 3   Prediction_P1     2075 non-null   object 
 4   Confidence_of_P1  2075 non-null   float64
 5   p1_dog            2075 non-null   bool   
 6   Prediction_P2     2075 non-null   object 
 7   Confidence_of_P2  2075 non-null   float64
 8   p2_dog            2075 non-null   bool   
 9   Prediction_P3     2075 non-null   object 
 10  Confidence_of_P3  2075 non-null   float64
 11  p3_dog            2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


Now, the columns have somehow meaninig names.

### 1.3.11 Twitter Archive Table: Columns names of 'doggo', 'floofer', 'pupper', and 'puppo' are actually values
### Define
Remove columns 'doggo', 'floofer', 'pupper', and 'puppo', and insert a column with name 'dog_stage'' and assign the value as the previous coulmn name.
### Code

In [44]:
df_arch_clean = df_arch_clean.replace('None','NaN')
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               1994 non-null   int64              
 1   in_reply_to_status_id  23 non-null     float64            
 2   in_reply_to_user_id    23 non-null     float64            
 3   timestamp              1994 non-null   datetime64[ns, UTC]
 4   source                 1994 non-null   object             
 5   text                   1994 non-null   object             
 6   expanded_urls          1994 non-null   object             
 7   rating_numerator       1994 non-null   int64              
 8   rating_denominator     1994 non-null   int64              
 9   name                   1994 non-null   object             
 10  doggo                  1994 non-null   object             
 11  floofer                1994 non-null   object           

In [45]:
df_arch_clean.insert(loc=8, column='dog_stage', value='NaN')

In [46]:
df_arch_clean

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,dog_stage,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,NaN,10,Phineas,NaN,NaN,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,NaN,10,Tilly,NaN,NaN,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,NaN,10,Archie,NaN,NaN,NaN,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,NaN,10,Darla,NaN,NaN,NaN,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,NaN,10,Franklin,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,https://twitter.com/dog_rates/status/666049248...,5,NaN,10,NaN,NaN,NaN,NaN,NaN
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,https://twitter.com/dog_rates/status/666044226...,6,NaN,10,a,NaN,NaN,NaN,NaN
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,https://twitter.com/dog_rates/status/666033412...,9,NaN,10,a,NaN,NaN,NaN,NaN
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30+00:00,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,https://twitter.com/dog_rates/status/666029285...,7,NaN,10,a,NaN,NaN,NaN,NaN


In [47]:
multi = 0
df_arch_clean.dog_stage = None
for i in df_arch_clean.index:
    multi = 0
    if (df_arch_clean.doggo[i] != 'NaN'):
        df_arch_clean.dog_stage[i] = 'doggo'
        multi = multi + 1
    if (df_arch_clean.floofer[i] != 'NaN'):
        df_arch_clean.dog_stage[i] = 'floofer'
        multi = multi + 1
    if (df_arch_clean.pupper[i] != 'NaN'):
        df_arch_clean.dog_stage[i] = 'pupper'
        multi = multi + 1
    if (df_arch_clean.puppo[i] != 'NaN'):
        df_arch_clean.dog_stage[i]= 'puppo'
        multi = multi + 1
    if multi > 11:
        df_arch_clean.dog_stage[i] = 'multiple'

<ipython-input-47-8953aa638463>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arch_clean.dog_stage[i] = 'doggo'
<ipython-input-47-8953aa638463>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arch_clean.dog_stage[i]= 'puppo'
<ipython-input-47-8953aa638463>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arch_clean.dog_stage[i] = 'pupper'
<ipython-input-47-8953aa638463>:9: SettingWithCopyWarning: 
A value is trying to be

In [48]:
df_arch_clean.drop(columns=['doggo','floofer','pupper','puppo'], inplace=True)
df_arch_clean['tweet_id'] = df_arch_clean['tweet_id'].astype('object')

### Test

In [49]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               1994 non-null   object             
 1   in_reply_to_status_id  23 non-null     float64            
 2   in_reply_to_user_id    23 non-null     float64            
 3   timestamp              1994 non-null   datetime64[ns, UTC]
 4   source                 1994 non-null   object             
 5   text                   1994 non-null   object             
 6   expanded_urls          1994 non-null   object             
 7   rating_numerator       1994 non-null   int64              
 8   dog_stage              306 non-null    object             
 9   rating_denominator     1994 non-null   int64              
 10  name                   1994 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(2), int64(2), object(

In [50]:
df_arch_clean.dog_stage.value_counts()

pupper     212
doggo       63
puppo       23
floofer      8
Name: dog_stage, dtype: int64

The 'dog_stage' column is present carrying the dog stage data, and the values are counted as presented above. Now, the table has only 11 columns

### 1.3.12 Clean the Dataset from tweets that have no images
### Define
Drop rows that have no images in the three tables.
### Code

In [51]:
image_index_new = df_arch_clean.tweet_id

In [52]:
df_predict_clean = df_predict_clean[df_predict_clean['tweet_id'].isin(image_index_new)]
df_api_clean = df_api_clean[df_api_clean['tweet_id'].isin(image_index_new)]

In [53]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               1994 non-null   object             
 1   in_reply_to_status_id  23 non-null     float64            
 2   in_reply_to_user_id    23 non-null     float64            
 3   timestamp              1994 non-null   datetime64[ns, UTC]
 4   source                 1994 non-null   object             
 5   text                   1994 non-null   object             
 6   expanded_urls          1994 non-null   object             
 7   rating_numerator       1994 non-null   int64              
 8   dog_stage              306 non-null    object             
 9   rating_denominator     1994 non-null   int64              
 10  name                   1994 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(2), int64(2), object(

In [54]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_id          1994 non-null   object 
 1   jpg_url           1994 non-null   object 
 2   img_num           1994 non-null   int64  
 3   Prediction_P1     1994 non-null   object 
 4   Confidence_of_P1  1994 non-null   float64
 5   p1_dog            1994 non-null   bool   
 6   Prediction_P2     1994 non-null   object 
 7   Confidence_of_P2  1994 non-null   float64
 8   p2_dog            1994 non-null   bool   
 9   Prediction_P3     1994 non-null   object 
 10  Confidence_of_P3  1994 non-null   float64
 11  p3_dog            1994 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 161.6+ KB


In [55]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2353
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   tweet_id         1994 non-null   int64
 1   retweet_count    1994 non-null   int64
 2   favorite_count   1994 non-null   int64
 3   followers_count  1994 non-null   int64
dtypes: int64(4)
memory usage: 77.9 KB


Now the 3 tables have 1994 rows in each table that have images and are original tweets.

### 1.3.13 The three tables have data that are related to the tweets, and these data should be in one table as all tables concern with the tweets’ stats and info
### Define
Merge all tables into one master table
### Code

In [56]:
twitter_archive_master = df_arch_clean.merge(df_predict_clean).merge(df_api_clean)

### Test

In [57]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 1993
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               1994 non-null   object             
 1   in_reply_to_status_id  23 non-null     float64            
 2   in_reply_to_user_id    23 non-null     float64            
 3   timestamp              1994 non-null   datetime64[ns, UTC]
 4   source                 1994 non-null   object             
 5   text                   1994 non-null   object             
 6   expanded_urls          1994 non-null   object             
 7   rating_numerator       1994 non-null   int64              
 8   dog_stage              306 non-null    object             
 9   rating_denominator     1994 non-null   int64              
 10  name                   1994 non-null   object             
 11  jpg_url                1994 non-null   object           

The merged "master" table has 1994 rows and only 25 columns

### 1.3.14 Saving the master dataset as 'twitter_archive_master.csv'

In [58]:
twitter_archive_master.to_csv('twitter_archive_master.csv')